In [ ]:
import pandas as pd
import seaborn as sns
import copy
import matplotlib
import plotly.express as px
from PIL import Image
import matplotlib.pyplot as plt
from pylab import rcParams
# Reglas de Asociación
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Análisis

In [ ]:
df_items = pd.read_csv("https://raw.githubusercontent.com/pokengineer/DataScience/main/datasets/coffee_items.csv")
df_orders = pd.read_csv("https://raw.githubusercontent.com/pokengineer/DataScience/main/datasets/coffee_orders.csv")

In [ ]:
df_items.head()

In [ ]:
df_orders.head()

In [ ]:
print(df_orders.shape[0])

In [ ]:
df_orders.order_id.duplicated().sum()

In [ ]:
df_orders[df_orders['quantity']<=0]

# columna de descripción

In [ ]:
df_items['descripcion'] =df_items.apply(lambda x: str(x['item_name']) if x['item_size']!= x['item_size'] else  str(x['item_name']) + ' | ' + str(x['item_size']) , axis=1)

In [ ]:
df_items.head(5)

In [ ]:
df_orders = pd.merge(df_orders,df_items[['item_id','descripcion']],on='item_id',how='left')

# reglas de asociación

In [ ]:
# Quitamos los ítems DOTCOM POSTAGE y POSTAGE porque no son productos.
# Llenamos con 0 los pedidos donde un producto no fué comprado
df_group = (df_orders.groupby(['order_id', 'descripcion'])['quantity'].sum().unstack().reset_index().fillna(0).set_index('order_id'))
# Seteamos True/False dependiendo de cada valor
df_group = df_group.applymap(lambda x: True if x >0 else False)
df_group

In [ ]:
x = 0.005
frequent_itemsets = apriori(df_group, min_support=x, use_colnames=True)
frequent_itemsets.sort_values(by="support", ascending=False)

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
# Ordenamos por confianza de mayor a menor
rules.sort_values(by="confidence", ascending=False).head(5)

# Consigna
- conceptualmente que representa la metrica 'support' y que representa la metrica 'confidence'
- obtener las reglas de asociacion usando item_name en lugar de la descripcion
- obtener las reglas de asociacion del siguiente dataset:<br>
http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

pueden usar pd.read_excel() para archivos xls o xlsx
